<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo12m_4FujiccoOkazu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os
import shutil

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [ ]:
output_dir = "runs_experiment"  # runs_experimentが既に存在している場合削除
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✅ ディレクトリ '{output_dir}' を削除しました。")
else:
    print(f"⚠️ ディレクトリ '{output_dir}' は存在しません。")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

In [ ]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

In [ ]:
model_path = "/content/drive/MyDrive/yolo12m.pt"
#model_path = "/content/drive/MyDrive/20250801iFujicco_item_yolo12m.pt" #継続学習の場合　名前は任意


In [ ]:
# YAML
import yaml

# YAMLファイルを読み込む
data_yaml = "/content/dataset/data.yaml"
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

In [ ]:
import os
from ultralytics import YOLO

def train_yolo(
    model_path,
    data_yaml,
    epochs=200,
    batch_size=8,          # per-GPU バッチ
    accumulate=4,          # 勾配蓄積 → 有効バッチ 8*4=32
    img_size=1280,
    output_dir="runs_experiment",
    iou_nms=0.95,          # 検証/推論側の内部NMSを緩く
    mosaic=0.15,
    mixup=0.05,
    copy_paste=0.6,
    close_mosaic=15,
    degrees=0.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.4,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.2, fliplr=0.5, flipud=0.0,
    lr0=0.008,             # 有効バッチ32前提で安定寄り
    optimizer="SGD",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,             # Colab/Windows安定
    project_name="okazu_yolov12m_1280_overlap"
):
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    os.makedirs(output_dir, exist_ok=True)
    print(f"Training YOLO12 with {epochs} epochs…")

    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        accumulate=accumulate,
        imgsz=img_size,

        # 学習率まわり
        optimizer=optimizer,
        lr0=lr0,
        cos_lr=cos_lr,
        warmup_epochs=warmup_epochs,

        # Aug/正則化（密集・重なり向け）
        mosaic=mosaic,
        mixup=mixup,
        copy_paste=copy_paste,
        close_mosaic=close_mosaic,
        degrees=degrees,
        shear=shear,
        perspective=perspective,
        translate=translate,
        scale=scale,
        hsv_h=hsv_h, hsv_s=hsv_s, hsv_v=hsv_v,
        erasing=erasing,
        fliplr=fliplr,
        flipud=flipud,

        # 検証時のNMS挙動（内部は緩くして外側で締める方針に合わせる）
        iou=iou_nms,

        # 実行系
        amp=amp,
        workers=workers,
        project=output_dir,
        name=project_name,
    )

    print(f"Training complete. Results saved to {output_dir}/{project_name}")

# ===== Colab 用パラメータ =====
model_path = "/content/drive/MyDrive/yolo12m.pt"
data_yaml  = "/content/dataset/data.yaml"

# A100 40GB なら batch=8, accumulate=4（有効32）で余裕
# T4 なら batch=4, accumulate=8（有効32）に下げてOK
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=300,
    batch_size=8,
    accumulate=4,
    img_size=1280,
    output_dir="runs_experiment",
    iou_nms=0.95,
    mosaic=0.15,
    mixup=0.05,
    copy_paste=0.6,
    close_mosaic=15,
    degrees=0.0, shear=0.0, perspective=0.0, translate=0.05, scale=0.4,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    erasing=0.2, fliplr=0.5, flipud=0.0,
    lr0=0.008,
    optimizer="SGD",
    cos_lr=True,
    warmup_epochs=3,
    amp=True,
    workers=0,
    project_name="okazu_yolov12m_1280_overlap"
)


In [ ]:
output_dir = "runs_experiment"
project_name = "okazu_yolov12m_1280_overlap"
best_model_path = f"{output_dir}/{project_name}/weights/best.pt"

if os.path.exists(best_model_path):
    print(f"Downloading: {best_model_path}")
    files.download(best_model_path)
else:
    print(f"[ERROR] best.pt が見つかりません: {best_model_path}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>